## Reading the Excel file, calculating the Rate of Returns (RoRs), and finding the average and covariances of RoRs.

In [9]:
import numpy as np
import pandas as pd
import math 

StocksData = pd.read_excel('/Users/mohamaddalati/Desktop/MGSC-662/Markowitz.xlsx', 
                           header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of return. (Because performance is sorted from earliest to latest date, we need to use -1)
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with)
rate_of_return_df.dropna(inplace = True)
rate_of_return_df

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
2016-06-01,-0.042370,-0.063923,-0.008266,-0.016583,-0.004754
2016-05-03,-0.001778,0.022290,0.000763,-0.013982,0.021170
2016-04-01,0.010850,0.007371,0.005007,0.017413,0.015098
2016-03-01,0.012760,0.049509,0.070753,0.080918,0.077503
2016-02-01,0.002186,-0.030895,0.003049,-0.039002,-0.001429
...,...,...,...,...,...
2011-05-02,-0.013163,-0.029379,-0.018793,-0.026757,-0.019635
2011-04-01,0.027264,0.067196,0.039839,0.015772,0.025772
2011-03-01,-0.014214,-0.031766,0.007638,-0.047549,0.024409
2011-02-01,0.022361,0.027530,0.028121,0.030477,0.054016


In [10]:
# Calculate the average monthly rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()
display(RoR_Avg)
# Calculate the variance for each stock 
RoR_Var = rate_of_return_df.var()
display(RoR_Var)
# To get the standard deviation or sigma for each stock 
RoR_Sigma = rate_of_return_df.std()
display(RoR_Sigma)
# Calculate the covariance between different stocks. 
RoR_Cov = rate_of_return_df.cov()
display(RoR_Cov)
# Turn the covariance dataframe into a NumPy matrix.
RoR_Cov_Mat = RoR_Cov.to_numpy()


print('For example, FTSE 100\'s (0th on the stock list) average RoR is:', RoR_Avg[0])

print('For example, the covariance between DAX and Russell 2000 RoRs is:', RoR_Cov_Mat[1, 4])

FTSE 100              0.000695
DAX                   0.006402
DJIA                  0.006931
DJ Asian Titans 50   -0.001075
Russell 2000          0.006875
dtype: float64

FTSE 100              0.001059
DAX                   0.002794
DJIA                  0.001082
DJ Asian Titans 50    0.002045
Russell 2000          0.002147
dtype: float64

FTSE 100              0.032549
DAX                   0.052861
DJIA                  0.032890
DJ Asian Titans 50    0.045223
Russell 2000          0.046334
dtype: float64

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
FTSE 100,0.001059,0.001299,0.000827,0.001012,0.000980
DAX,0.001299,0.002794,0.001224,0.001597,0.001700
DJIA,0.000827,0.001224,0.001082,0.001073,0.001293
DJ Asian Titans 50,0.001012,0.001597,0.001073,0.002045,0.001405
Russell 2000,0.000980,0.001700,0.001293,0.001405,0.002147


For example, FTSE 100's (0th on the stock list) average RoR is: 0.0006948773946744407
For example, the covariance between DAX and Russell 2000 RoRs is: 0.001700419240178296


# Scenario 1a
Find the Optimal portfolio that minimizes the portfolio risk subject to non-negative return. What are the optimal solution and value?

In [14]:
import gurobipy as gb
from gurobipy import *

# Get data 
Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

Monthly_RoR = [RoR_Avg[0], RoR_Avg[1], RoR_Avg[2], RoR_Avg[3], RoR_Avg[4]]

Variance = [RoR_Var[0], RoR_Var[1], RoR_Var[2], RoR_Var[3], RoR_Var[4]]

SD_Return = [RoR_Sigma[0], RoR_Sigma[1], RoR_Sigma[2], RoR_Sigma[3], RoR_Sigma[4]]

Covariance = RoR_Cov_Mat

# Quantify
R = len(Stock_Name) # 5 

[[0.00105943 0.00129948 0.00082734 0.00101192 0.00098002]
 [0.00129948 0.00279426 0.00122365 0.00159699 0.00170042]
 [0.00082734 0.00122365 0.00108175 0.00107308 0.00129305]
 [0.00101192 0.00159699 0.00107308 0.00204513 0.00140453]
 [0.00098002 0.00170042 0.00129305 0.00140453 0.00214685]]


In [16]:
list = [ (0, 1),
         (0, 2),
         (0, 3),
         (0, 4), 
         (1, 2),
         (1, 3), 
         (1, 4), 
         (2, 3),
         (2, 4),
         (3, 4)]

l = len(list) # 10


# model initialization 
model = gb.Model('Risk Portfolio Optimization')

# decision variables 
X = model.addVars(R, lb=0, ub=1, vtype=GRB.CONTINUOUS, 
                  name = ['% of money invested in stock_'+i for i in Stock_Name])

# objective function 
model.setObjective(sum( X[i]**2 * Variance[i] for i in range(R))
                       +
                sum(2*X[i[0]]*X[i[1]]*Covariance[i[0],i[1]] for i in list ), GRB.MINIMIZE)

# constraints 
model.addConstr( sum(X[i] for i in range(R) ) == 1 ) # summation of fraction of investments == 100%
model.addConstr( sum(X[i]*Monthly_RoR[i] for i in range(R)) >= 0) # expected return 

# solve 
model.optimize()
print('Portfolio risk is', math.sqrt(model.objVal))
print('')

# to get variables 
for i in model.getVars():
    print(i.varName, '=', round(i.x*100,3))


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x0daebafd
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [7e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-03, 7e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 2 rows, 5 columns, 10 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.500e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.47487650e+05 -2.47487650e+05  5.00e+03 1.09e-06  2.98e+05     0s
   1   5.68608190e+03 -5.72792175e+03  4.20e+0

# Scenario 1b 
If the initial allocation is 20% in each index and changing the position requires incurring transaction costs. 
Find the optimal portfolio to minimize risk subject to non-negative return.

In [18]:
list = [ (0, 1),
         (0, 2),
         (0, 3),
         (0, 4), 
         (1, 2),
         (1, 3), 
         (1, 4), 
         (2, 3),
         (2, 4),
         (3, 4)]

l = len(list)

# model initialization 
model2 = gb.Model('Risk Portfolio Optimization_1b')

# decision variables 
X = model2.addVars(R, lb=0, ub=1, vtype=GRB.CONTINUOUS, 
                  name = ['% of money invested in stock_'+i for i in Stock_Name])

# objective function 
model2.setObjective(sum( X[i]**2 * Variance[i] for i in range(R))
                       +
                sum(2*X[i[0]]*X[i[1]]*Covariance[i[0],i[1]] for i in list ), GRB.MINIMIZE)

# constraints 
model2.addConstr( sum(X[i] for i in range(R) ) == 1 ) # summation of fraction of investments == 100%
model2.addConstr( sum(X[i]*Monthly_RoR[i] - (X[i]-0.2)**2 for i in range(R)) >= 0) # expected return 

# solve 
model2.optimize()
print(' risk of the portfolio is ', math.sqrt(model2.objVal))
print('')

# to get variables 
for i in model2.getVars():
    print(i.varName, '=', round(i.x*100,3))


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x83609702
Model has 15 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [4e-01, 4e-01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-03, 7e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e-01, 2e-01]
Presolve time: 0.01s
Presolved: 19 rows, 14 columns, 45 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 1.000e+02
 Factor NZ  : 2.100e+02
 Factor Ops : 2.870e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 